In [1]:
#/usr/bin/env python

import sqlite3
from sqlite3 import Error
import unittest
import datetime
from datetime import timedelta
import pandas as pd
from pandas import Series, DataFrame
import pandas_datareader.data as web
global conn



In [2]:
def get50dayavg(conn):
  
    cur = conn.cursor()
    cur.execute("SELECT * FROM (SELECT * FROM stockprices order by id DESC limit 5) order by id")
    c = len(cur.fetchall()) 
            
    if c < 5:
        return -1 
            
    elif c == 5:
        cur = conn.cursor()
        c = cur.execute("SELECT sum(volume)/count(*) FROM stockprices daysavgvolume")
        for row in c:
            return row[0]

    
class Test_setup(unittest.TestCase):
    
    def test_get50dayavaerage_1(self):
        
        database = "/Users/Ravi/Downloads/stock_backend/stocksqlite.db"
        conn = create_connection(database)
        
        cur = conn.cursor()
        cur.execute('DELETE from stockprices')
           
        
        for i in range (0,7):
            sql = ''' INSERT INTO stockprices(date,stockdetails_id,openvalue,highvalue,lowvalue,
                       closevalue,adjclose,volume,daysavgvolume)
                       VALUES(Date('now'),1,3,3,3,4,4,20000000,0) '''
            
            cur = conn.cursor()
            cur.execute(sql)
        
        cur.execute("SELECT * FROM (SELECT * FROM stockprices order by id DESC limit 5) order by id")
        
        rows = cur.fetchall()
 
        for row in rows:
          print(row)
                    
        result = get50dayavg(conn)

        self.assertEqual(20000000, result)
  
    def test_get50dayavaerage_2(self):
        
        database = "/Users/Ravi/Downloads/stock_backend/stocksqlite.db"
        conn = create_connection(database)
        
        cur = conn.cursor()
        cur.execute('DELETE from stockprices')
        
        for i in range (0,3):
            sql = ''' INSERT INTO stockprices(date,stockdetails_id,openvalue,highvalue,lowvalue,
                    closevalue,adjclose,volume,daysavgvolume) VALUES(Date('now'),1,3,3,3,4,4,10,0) '''
            
            cur = conn.cursor()
            cur.execute(sql)
        
        cur.execute("SELECT * FROM stockprices")
 
        rows = cur.fetchall()
 
        for row in rows:
          print(row)
                    
        result = get50dayavg(conn)

        self.assertEqual(-1, result)
   
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
            

EE
ERROR: test_get50dayavaerage_1 (__main__.Test_setup)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-afdb7f3998a4>", line 22, in test_get50dayavaerage_1
    conn = create_connection(database)
NameError: name 'create_connection' is not defined

ERROR: test_get50dayavaerage_2 (__main__.Test_setup)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-afdb7f3998a4>", line 50, in test_get50dayavaerage_2
    conn = create_connection(database)
NameError: name 'create_connection' is not defined

----------------------------------------------------------------------
Ran 2 tests in 0.008s

FAILED (errors=2)


In [5]:
def get_symbol_data_today(now,sym):
    
    #start=%s/%s/%s" % (i.day, i.month, i.year)
    prev = datetime.datetime.now() - timedelta(days=3)
    start=datetime.datetime(prev.year,prev.month,prev.day)
    end=datetime.datetime(now.year,now.month,now.day)
    try:
        f = web.DataReader(sym, 'yahoo', start,end)
    except Error as e:
        print(e)    
    
    #f=f.set_index(['Date'])
    #r=web.get_quote_yahoo('AAPL')
    return f.tail(1)

def update_data_to_db(conn,sym):
    sym="NVDA"
    now = datetime.datetime.now()
    data = get_symbol_data_today(now,sym)
    #print(data.keys())
    t=data['Open'].values[0]
    t1=data['High'].values[0]
    t2=data['Low'].values[0]
    t3=data['Close'].values[0]
    t4=data['Adj Close'].values[0]
    t5=data['Volume'].values[0]
       
    symbolid = get_symbol_id(conn, sym)
    avg = get50dayavg(conn)
    
    stockpricesinfo = (datetime.date.today(),symbolid,t,t1,t2,t3,t4,t5,avg)
    id = insert_stockprices(conn, stockpricesinfo)
    print_stockprices(conn)
    
def get_symbol_id(conn,sym):
    
    cur = conn.cursor()
    cur.execute("SELECT id FROM stockdetails WHERE companysymbol=?",(sym,))
    row=cur.fetchone()
    print(row[0])    
    return row[0] 

   
def insert_stockprices(conn,stockpricesinfo):    
    """
    Create a new stockprices into the stockprices table
    :param conn:
    :param stockprices:
    :return: id
    """
    cur = conn.cursor()
    count = cur.execute("SELECT count(*) FROM stockprices WHERE date = ?",(datetime.date.today(),))
    
    for row in count:
        
        print("count rows :",row[0])
        if (row[0] == 0):
            sql = ''' INSERT INTO stockprices(date,stockdetails_id,openvalue,highvalue,lowvalue,
                      closevalue,adjclose,volume,daysavgvolume) VALUES(?,?,?,?,?,?,?,?,?) ''' 
            cur.execute(sql, stockpricesinfo)
                 
        else:
            sql1 = ''' UPDATE stockprices SET date = ?,stockdetails_id = ?,openvalue=?,highvalue=?,
                      lowvalue=?,closevalue=?,adjclose=?,volume=?,daysavgvolume=? '''
            cur.execute(sql1, stockpricesinfo)
            
    c = cur.execute("SELECT volume FROM stockprices")
            
    for row in c:
        c1 = int.from_bytes(row[0], byteorder='little')
        cur.execute("UPDATE stockprices SET volume = ?", (c1,))
                
    #return cur.lastrowid 

def print_stockprices(conn):
    """
    Query all rows in the stockprices table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM stockprices")    
    
    rows = cur.fetchall()
    for row in rows:
        print(row)
    
        
        
def create_connection(db):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db)
        print("Database opened successfully")
        return conn
    except Error as e:
        print(e)
 
    return None

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def close_connection(db_conn):
    """ Close connection to the database """
    db_conn.close()   
    
def insert_stockdetails(conn, stockinfo):
    """
    Create a new stockdetails into the stockdetails table
    :param conn:
    :param stockdetails:
    :return: id
    """
    sql = ''' INSERT INTO stockdetails(companysymbol,companyname)
              VALUES(?,?) '''
    
    cur = conn.cursor()
    cur.execute(sql, stockinfo)
    
    return cur.lastrowid 

                
def print_stockdetails(conn):
    """
    Query all rows in the stockdetails table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM stockdetails")
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
                
               
def main():
    
       
    database = "/Users/Ravi/Downloads/stock_backend/stocksqlite.db"
 
    stockdetails_table = """CREATE TABLE IF NOT EXISTS stockdetails (
                                        id integer PRIMARY KEY,
                                        companysymbol varchar(10) NOT NULL,
                                        companyname text NOT NULL
                                        
                                    );"""                            
                                    
 
    stockprices_table = """CREATE TABLE IF NOT EXISTS stockprices (
                                    id integer PRIMARY KEY,
                                    date date NOT NULL,
                                    stockdetails_id integer NOT NULL,
                                    openvalue real NOT NULL,
                                    highvalue real NOT NULL,
                                    lowvalue real NOT NULL,
                                    closevalue real NOT NULL,
                                    adjclose real NOT NULL,
                                    volume BIGINT NOT NULL,
                                    daysavgvolume real NOT NULL,
                                    FOREIGN KEY (stockdetails_id) REFERENCES stockdetails (id)
                                );"""
   
 
    # create a database connection
    conn = create_connection(database)
    if conn is not None:
        # create stockdetails table
        create_table(conn, stockdetails_table)
        
        # create stockprices table
        create_table(conn, stockprices_table)
    else:
        print("Error! cannot create the database connection.")

    with conn:
        # create a new stockdetails
        
        #stockinfo = ('NVDA', 'Nvidia Corporataion');
        #id = insert_stockdetails(conn, stockinfo)
        print_stockdetails(conn)
        
        #while loop - get symbols from st
        # create a new stockprices
        sym = "NVDA"
        stockprices = update_data_to_db(conn, sym)
        
    # close database connection      
    close_connection(conn)

if __name__ == '__main__':
    main()


Database opened successfully
(1, 'NVDA', 'Nvidia Corporataion')


RemoteDataError: Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/NVDA?period1=1516003200&period2=1516348799&interval=1d&events=history&crumb=%5Cu002FnRvPH2ja6d